This is purely experiemental by testing state space model ideas from [Vihola, Helske, Franks \(2020\)](https://onlinelibrary.wiley.com/doi/10.1111/sjos.12492). Stan source code can be found [here](https://github.com/helske/walker). Although the main idea for the article is on the importance sampling, which is not tested yet in this notebook, we still use the process to compare unsmoothed states estimation for study purpose.

In [145]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
multiprocessing.set_start_method("fork", force=True)

from orbit.utils.stan import get_compiled_stan_model_simplified, compile_stan_model_simplified
from orbit.utils.dataset import load_iclaims
import arviz as az

In [148]:
raw_data = load_iclaims()
y = raw_data['claims']
y = (y - np.mean(y))/np.std(y)

In [149]:
def make_fourier_series(n, period, order=3, shift=0):
    t = np.arange(1, n + 1) + shift
    out = list()
    for i in range(1, order + 1):
        x = 2.0 * i * np.pi * t / period
        out.append(np.cos(x))
        out.append(np.sin(x))
    out = np.column_stack(out)
    return out

In [150]:
intercept_reg = np.ones((len(y), 1))
fs_reg = make_fourier_series(len(y), 52, order=3)
xreg = np.concatenate([intercept_reg, fs_reg], -1)
m = xreg.shape[1]
a1 = np.array([y[1]] + [0.0] * (m-1))
p1 = np.zeros_like(a1)

In [ ]:
walker_model_path = "./stan/walker.stan"
compiled_path = compile_stan_model_simplified(walker_model_path)
walker_mod = get_compiled_stan_model_simplified(compiled_path)
del compiled_path

In [ ]:
data = {
    'n' : len(y),
    'k' : 1,
    'xreg' : xreg,
    'y': y,
    'beta_mean' : np.zeros((1)),
    'beta_sd' : np.ones((1)),
    'sigma_mean': np.ones((2, )),
    'sigma_sd': np.ones((2, )),
    'n_new': 0,
    'xreg_new': np.ones((1, 0)),
}

In [ ]:
walker_fit = walker_mod.sampling(
    data=data,
    warmup=4000,
    iter=5000,
    chains=4,
)


Gradient evaluation took 0.001652 seconds
1000 transitions using 10 leapfrog steps per transition would take 16.52 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.001507 seconds
1000 transitions using 10 leapfrog steps per transition would take 15.07 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.001309 seconds
1000 transitions using 10 leapfrog steps per transition would take 13.09 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.001273 seconds
1000 transitions using 10 leapfrog steps per transition would take 12.73 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 5000 [  0%]  (Warmup)
Iteration:    1 / 5000 [  0%]  (Warmup)
Iteration:    1 / 5000 [  0%]  (Warmup)
Iteration:    1 / 5000 [  0%]  (Warmup)


Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: Exception: multiply: A[1] is nan, but must not be nan!  (in 'unknown file name' at line 19)
  (in 'unknown file name' at line 99)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: Exception: multiply: A[1] is nan, but must not be nan!  (in 'unknown file name' at line 19)
  (in 'unknown file name' at line 99)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informationa

Iteration:  500 / 5000 [ 10%]  (Warmup)
Iteration:  500 / 5000 [ 10%]  (Warmup)
Iteration:  500 / 5000 [ 10%]  (Warmup)
Iteration:  500 / 5000 [ 10%]  (Warmup)
Iteration: 1000 / 5000 [ 20%]  (Warmup)
Iteration: 1000 / 5000 [ 20%]  (Warmup)
Iteration: 1000 / 5000 [ 20%]  (Warmup)
Iteration: 1000 / 5000 [ 20%]  (Warmup)
Iteration: 1500 / 5000 [ 30%]  (Warmup)
Iteration: 1500 / 5000 [ 30%]  (Warmup)
Iteration: 1500 / 5000 [ 30%]  (Warmup)
Iteration: 1500 / 5000 [ 30%]  (Warmup)
Iteration: 2000 / 5000 [ 40%]  (Warmup)
Iteration: 2000 / 5000 [ 40%]  (Warmup)
Iteration: 2000 / 5000 [ 40%]  (Warmup)
Iteration: 2000 / 5000 [ 40%]  (Warmup)
Iteration: 2500 / 5000 [ 50%]  (Warmup)
Iteration: 2500 / 5000 [ 50%]  (Warmup)
Iteration: 2500 / 5000 [ 50%]  (Warmup)
Iteration: 2500 / 5000 [ 50%]  (Warmup)
Iteration: 3000 / 5000 [ 60%]  (Warmup)
Iteration: 3000 / 5000 [ 60%]  (Warmup)
Iteration: 3000 / 5000 [ 60%]  (Warmup)
Iteration: 3000 / 5000 [ 60%]  (Warmup)
Iteration: 3500 / 5000 [ 70%]  (Warmup)


Iteration: 5000 / 5000 [100%]  (Sampling)

 Elapsed Time: 26.4267 seconds (Warm-up)
               7.66955 seconds (Sampling)
               34.0962 seconds (Total)



In [ ]:
az_posteriors = az.from_pystan(walker_fit)
# converging
az.plot_trace(az_posteriors, var_names=['sigma_b', 'sigma_y'], compact=False, figsize=(24, 12));

In [ ]:
posteriors = walker_fit.extract(permuted=True)

In [ ]:
states_mean = posteriors['y_rep']
states_mean = np.mean(states_mean, 0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))
x = np.arange(0, len(y))
ax.scatter(x, y, c='grey')
ax.plot(x, states_mean, color='blue');